# md to ipynb

## jupytext

In [ ]:
! pip install jupytext


In [4]:
response = requests.get("https://content.codecademy.com/courses/learn-html-forms/submit%20button2.png")

In [8]:
response.content

b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00V\x00\x00\x008\x08\x00\x00\x00\x00B\xc3P\x01\x00\x00\x18QiCCPICC Profile\x00\x00X\x85\x95Y\t4U\xdd\xdb\xdf\xe7\xce\xf7\x9a\xef5\xcf\xf3L\xe6y\x9e\xe7y&\xe1\x9a\xae)\xae1\x94H2\x95\xa8\x10\xa5\x92\x8c\x95J\xa1\x12\x91\x06\xa5\x0c\xbd$I$C\xa5PQ\x19\xf2\x1dC\xfd\xdf\xff\xfb\xae\xf5}\xeb\xdbk\x9ds~\xe7\xd9\xcf~\x86\xbd\x9f=<\xe7\x00\xc0\xb5\xdf722\x0c\xc1\x08@xD\x0c\xd5\xc1\xd4\x80\xdf\xcd\xdd\x83\x1f\xfb\x16\x10\x00\t\x00\xc0\x08$}\xc9\xd1\x91\xfavvV\xf0\x1b\xf8\xfd\xfc\xef\xb24\x08\xa0\x8d\xe7s\x99\rY\xff\xae\xff_\x0b\x93\x7f@4\x19\x00\xc8\x0e\xc6~\xfe\xd1\xe4p\x18_\x07\x00\x95N\x8e\xa4\xc6\x00\x80Q\x83\xe9B\xf11\x91\x1b\xd8\x0b\xc6\xccT\xd8@\x18Gn\xe0\xa0-\x9c\xbe\x81\xfd\xb6p\xf1&\x8f\x93\x83!\x8c/\x02\x80\xa3\xf5\xf5\xa5\x06\x01@\xdf\x0c\xd3\xf9\xe3\xc8A\xb0\x1c\xfa!\xb8\x8e\x18\xe1O\x89\x80Yga\xacC\x0e\xf6\xf5\x07\x80K\x1a\xe6\x91\x0e\x0f\xdf\xbd\x81\xdd`,\xee\xf779A\xff%\xd3\xef\x8fL_\xdf\xa0?x\xcb\x97\xcd\x823\xa2DG\x86\xf9\xee\xf9\x

In [10]:
# NOTICE: make sure the picture could be shown in the md file
import os
import re
import base64
import nbformat as nbf
from pathlib import Path
import requests
# Replace 'my_note_database.md' with the path to your markdown file

def transform_md_to_ipynb(file_name, folder_name= 'files/md_to_ipynb',transform_link = True):
    markdown_file_dir = f'{folder_name}/{file_name}.md'
    # Replace 'output_notebook.ipynb' with the desired output file name
    output_notebook = f'.ipynb'
    # Run the jupytext command
    os.system(f'jupytext --to {output_notebook} {markdown_file_dir}')

    def transform_link_to_attachment(folder_name, file_name):
        input_dir = f'{folder_name}/{file_name}.ipynb'
        output_dir =  f'{folder_name}/{file_name}_attach.ipynb'
        # Load the existing notebook
        with open(input_dir, "r") as f:
            nb = nbf.read(f, as_version=4)

        # Regular expression to match image links
        image_pattern = re.compile(r'!\[.*?\]\((.+?)\)')

        # Iterate through cells in the notebook
        new_cells = []
        for cell in nb.cells:
            if cell.cell_type == "markdown":
                # Search for image links in the Markdown cell
                matches = list(image_pattern.finditer(cell.source))
                if matches:
                    last_end = 0
                    for match in matches:
                        # Add the content before the image as a separate Markdown cell
                        content_before_image = cell.source[last_end:match.start()]
                        if content_before_image.strip():
                            new_cells.append(nbf.v4.new_markdown_cell(content_before_image))

                        image_path = match.group(1)
                        image_name = Path(image_path).name
                        image_name = "".join(char if char.isalnum() else "_" for char in image_name)
                        if image_path.endswith('.svg'):
                            if image_path.startswith("http"):
                                response = requests.get(image_path)
                                svg_data = response.content
                            else:
                                with open(f'{folder_name}/{image_path}', "r") as img_file:
                                    svg_data = img_file.read()
                            svg_base64 = base64.b64encode(svg_data.encode("utf-8")).decode("utf-8")
                            attachment_cell = nbf.v4.new_markdown_cell()
                            attachment_cell.source = f"![{image_name}](attachment:{image_name})"
                            attachment_cell.attachments = {
                                image_name: {
                                    "image/svg+xml": svg_base64
                                }   
                            }
                        else:
                        # Create a new Markdown cell with the image as an attachment
                            if image_path.startswith("http"):
                                # Download image from the URL
                                response = requests.get(image_path)
                                img_data = response.content

                                # Read image file

                            else:
                                with open(f'{folder_name}/{image_path}', "rb") as img_file:
                                    img_data = img_file.read()

                            img_base64 = base64.b64encode(img_data).decode("utf-8")
                            img_mime = f"image/{Path(image_path).suffix[1:]}"
                            attachment_cell = nbf.v4.new_markdown_cell()
                            attachment_cell.source = f"![{image_name}](attachment:{image_name})"
                            attachment_cell.attachments = {
                                image_name: {
                                    img_mime: img_base64
                                }
                            }
                        new_cells.append(attachment_cell)
                        last_end = match.end()

                    # Add the content after the last image as a separate Markdown cell
                    content_after_image = cell.source[last_end:]
                    if content_after_image.strip():
                        new_cells.append(nbf.v4.new_markdown_cell(content_after_image))
                else:
                    new_cells.append(cell)
            else:
                new_cells.append(cell)

        nb.cells = new_cells

        # Save the modified notebook
        with open(output_dir, "w") as f:
            nbf.write(nb, f)

    if transform_link:
        transform_link_to_attachment(folder_name, file_name)
folder_path = 'files/md_to_ipynb'
for file in os.listdir(folder_path):
    if file.endswith('.md'):
        transform_md_to_ipynb(file.split('.')[0])


In [9]:
from general_function import delete_all_files
# Usage

delete_all_files(folder_path, reserve_dir=['typora-user-images'])

# ipynb to md

**First, run command**

In [2]:
!jupyter nbconvert --to markdown --config config.py files/ipynb_to_md/review.ipynb

{'NotebookApp': {'contents_manager_class': 'jupyter_server.services.contents.largefilemanager.LargeFileManager'}, 'Exporter': {'preprocessors': ['extract_attachments_preprocessor.ExtractAttachmentsPreprocessor']}}

[NbConvertApp] Converting notebook files/ipynb_to_md/review.ipynb to markdown
[NbConvertApp] Support files will be in review_files\
[NbConvertApp] Making directory files/ipynb_to_md\review_files
[NbConvertApp] Making directory files/ipynb_to_md\review_files
[NbConvertApp] Making directory files/ipynb_to_md\review_files
[NbConvertApp] Making directory files/ipynb_to_md\review_files
[NbConvertApp] Making directory files/ipynb_to_md\review_files
[NbConvertApp] Making directory files/ipynb_to_md\review_files
[NbConvertApp] Making directory files/ipynb_to_md\review_files
[NbConvertApp] Making directory files/ipynb_to_md\review_files
[NbConvertApp] Making directory files/ipynb_to_md\review_files
[NbConvertApp] Making directory files/ipynb_to_md\review_files
[NbConvertApp] Making directory files/ipynb_to_md\review_files
[NbConvertApp] Making directory files/ipynb_to_md\review_files
[NbConvertApp] Making directory files/ipynb_to_md\review_files
[NbConvertApp] Making directory files/ipynb_to_md

**Finally, clear the folder for the next usage**

In [3]:
from general_function import delete_all_files
# Usage
folder_path = 'files/ipynb_to_md/'
delete_all_files(folder_path)

# txt to ipynb

In [1]:
! pip install nbformat

In [5]:
import nbformat as nbf
import os
folder_path = 'files/txt_to_ipynb'
def txt_to_ipynb(txt_file, ipynb_file):
    # Read the .txt file
    with open(txt_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    # Create a new notebook
    nb = nbf.v4.new_notebook()

    # Iterate over the lines in pairs and add them to the notebook
    for i in range(0, len(lines), 2):
        # Take two lines at a time
        text = lines[i:i+2]
        # Convert list of lines to a single string
        text = '\n综述：'.join(text)
        text += '\n模型图：\n\n案例数据'
        # Create a new markdown cell
        cell = nbf.v4.new_markdown_cell(text)
        # Add the cell to the notebook
        nb['cells'].append(cell)

    # Write the notebook to a .ipynb file
    with open(ipynb_file, 'w', encoding='utf-8') as f:
        nbf.write(nb, f)

# Example usage
for f in os.listdir(folder_path):
    if f.endswith('.txt'):
        txt_to_ipynb(f'{folder_path}/{f}', f'{folder_path}/{f.split(".")[0]}.ipynb')


# ipynb modification

## add citation

In [4]:
import json
import os
citation = "Your citation text here."


input_folder = 'files/ipynb'
for file in os.listdir(input_folder):
    with open(f'{input_folder}/{file}', 'r') as f:
        notebook = json.load(f)
    # Iterate through the cells and add a citation to each markdown cell
    for cell in notebook['cells']:
        if cell['cell_type'] == 'markdown':
            source = cell['source']
            if not any(line.startswith('#') for line in source) and not any('citation' in line for line in source):
                # Add citation at the end of the markdown cell
                new_source = [].append(source)
                cell['source'].insert(0, f'> {citation}\n\n')

    # Save the updated ipynb file
    with open(f'{input_folder}/updated_{file}', 'w') as f:
        json.dump(notebook, f, indent=1)


In [6]:
from general_function import delete_all_files
delete_all_files(input_folder)